In [1]:
import os, sys
from PIL import Image
import cv2
import pandas as pd
import numpy as np

In [2]:
class BOW(object):
    
    def __init__(self,):
        #建立SIFT物件提取關鍵點
        self.feature_detector  = cv2.xfeatures2d.SIFT_create()
        self.descriptor_extractor = cv2.xfeatures2d.SIFT_create()

    def path(self,cls,i):
        filenames = os.listdir(self.train_path + "/" + cls) 
        return '%s/%s/%s'%(self.train_path,cls,filenames[i])

    def fit(self,train_path,k):
        
        self.train_path = train_path        
        
        #FLANN匹配
        flann_params = dict(algorithm=1,tree=5)
        flann = cv2.FlannBasedMatcher(flann_params,{})
        
        #BOW訓練器
        bow_kmeans_trainer = cv2.BOWKMeansTrainer(k)
        
               
        for j in range(50):
            try:
                DIR = train_path + '/' + classname[j]
                dirlen = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
            except:
                    print("error:", sys.exc_info()[0])
            for i in range(200):
                try:
                    bow_kmeans_trainer.add(self.sift_descriptor_extractor(self.path(classname[j],i)))
                except:
                    print("error:", sys.exc_info()[0])
        
        #k-means聚類
        voc = bow_kmeans_trainer.cluster()
        
        print(type(voc),voc.shape)
        
        #bow提取
        self.bow_img_descriptor_extractor = cv2.BOWImgDescriptorExtractor(self.descriptor_extractor,flann)        
        self.bow_img_descriptor_extractor.setVocabulary(voc)
                

        traindata,trainlabels = [],[]
        for n in range(50):     
            try:
                DIR = train_path + '/' + classname[n]
                dirlen = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
            except:
                    print("error:", sys.exc_info()[0])
            for i in range(200):   
                try:
                    traindata.extend(self.bow_descriptor_extractor(self.path(classname[n],i)))
                    trainlabels.append(n)
                except:
                    print("error:", sys.exc_info()[0])
        
    
        #建立SVM進行訓練
        self.svm = cv2.ml.SVM_create()
        self.svm.train(np.array(traindata),cv2.ml.ROW_SAMPLE,np.array(trainlabels))

    def predict(self,img_path):
        data = self.bow_descriptor_extractor(img_path)
        res = self.svm.predict(data)
             
        print(img_path,'\t',res[1][0][0])

        for i in range(50):
            if res[1][0][0] == i:
                return i
    
    def sift_descriptor_extractor(self,img_path):
        im = cv2.imread(img_path,0)     
        return self.descriptor_extractor.compute(im,self.feature_detector.detect(im))[1]  
     
    def bow_descriptor_extractor(self,img_path):
        im = cv2.imread(img_path,0)
        return self.bow_img_descriptor_extractor.compute(im,self.feature_detector.detect(im))

In [3]:
f = open(r'test.txt')
f2 = open(r'train.txt')
f3 = open(r'val.txt')

test_num = 0
test = []
test_origin = []

train_num = 0
train = []
train_origin = []

val_num = 0
val = []
val_origin = []

for line in f:
    line = line.replace("images", "images/test")
    line2 = line.split(" ")
    test.append(line2[0])
    test_origin.append(line2[1])
    test_num += 1

for line in f2:
    line = line.replace("images", "train/test")
    line2 = line.split(" ")
    train.append(line2[0])
    train_origin.append(line2[1])
    train_num += 1
    
for line in f3:
    line = line.replace("images", "images/val")
    line2 = line.split(" ")
    val.append(line2[0])
    val_origin.append(line2[1])
    val_num += 1

ImagesPath = './images'
allFolderList = os.listdir('./images')
n = 0;
classname = [];
for folder in allFolderList:
    if os.path.isdir(os.path.join(ImagesPath,folder)):
        classname.append(folder)
        n += 1
        
del(classname[n-1])
del(classname[n-2])

train_path = './images'
DIR = train_path + '/' + classname[0]
dirlen = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]) 

In [4]:
bow = BOW()
bow.fit(train_path , 40)

pre_test_samples = test_num
pre_test = np.zeros(pre_test_samples,dtype=np.int)

pre_val_samples = val_num
pre_val = np.zeros(pre_val_samples,dtype=np.int)

<class 'numpy.ndarray'> (40, 128)


In [5]:
#預測
for index in range(test_num):  
    pre_test[index] = bow.predict(test[index])

for index in range(val_num):
    pre_val[index] = bow.predict(val[index])

images/test/n02111277/n02111277_9420.JPEG 	 19.0
images/test/n02111277/n02111277_9422.JPEG 	 13.0
images/test/n02111277/n02111277_9484.JPEG 	 4.0
images/test/n02111277/n02111277_951.JPEG 	 36.0
images/test/n02111277/n02111277_9518.JPEG 	 36.0
images/test/n02111277/n02111277_9562.JPEG 	 28.0
images/test/n02111277/n02111277_9619.JPEG 	 5.0
images/test/n02111277/n02111277_9622.JPEG 	 19.0
images/test/n02111277/n02111277_9641.JPEG 	 19.0
images/test/n02111500/n02111500_9386.JPEG 	 33.0
images/test/n02111500/n02111500_9398.JPEG 	 45.0
images/test/n02111500/n02111500_9403.JPEG 	 5.0
images/test/n02111500/n02111500_9405.JPEG 	 40.0
images/test/n02111500/n02111500_9409.JPEG 	 28.0
images/test/n02111500/n02111500_9410.JPEG 	 27.0
images/test/n02111500/n02111500_9418.JPEG 	 29.0
images/test/n02111500/n02111500_9420.JPEG 	 28.0
images/test/n02111500/n02111500_9423.JPEG 	 36.0
images/test/n02111889/n02111889_9654.JPEG 	 26.0
images/test/n02111889/n02111889_97.JPEG 	 3.0
images/test/n02111889/n0211

images/test/n02115913/n02115913_9916.JPEG 	 22.0
images/test/n02115913/n02115913_9923.JPEG 	 33.0
images/test/n02116738/n02116738_9633.JPEG 	 18.0
images/test/n02116738/n02116738_9673.JPEG 	 33.0
images/test/n02116738/n02116738_9710.JPEG 	 19.0
images/test/n02116738/n02116738_9748.JPEG 	 10.0
images/test/n02116738/n02116738_9762.JPEG 	 31.0
images/test/n02116738/n02116738_9769.JPEG 	 18.0
images/test/n02116738/n02116738_9782.JPEG 	 33.0
images/test/n02116738/n02116738_9798.JPEG 	 19.0
images/test/n02116738/n02116738_9808.JPEG 	 18.0
images/test/n02117135/n02117135_9800.JPEG 	 18.0
images/test/n02117135/n02117135_9824.JPEG 	 33.0
images/test/n02117135/n02117135_983.JPEG 	 15.0
images/test/n02117135/n02117135_9840.JPEG 	 18.0
images/test/n02117135/n02117135_9848.JPEG 	 37.0
images/test/n02117135/n02117135_9852.JPEG 	 20.0
images/test/n02117135/n02117135_9855.JPEG 	 41.0
images/test/n02117135/n02117135_9857.JPEG 	 18.0
images/test/n02117135/n02117135_9866.JPEG 	 37.0
images/test/n02119022

images/test/n02130308/n02130308_9809.JPEG 	 37.0
images/test/n02130308/n02130308_9836.JPEG 	 33.0
images/test/n02132136/n02132136_9775.JPEG 	 13.0
images/test/n02132136/n02132136_9792.JPEG 	 30.0
images/test/n02132136/n02132136_9797.JPEG 	 15.0
images/test/n02132136/n02132136_9807.JPEG 	 26.0
images/test/n02132136/n02132136_981.JPEG 	 31.0
images/test/n02132136/n02132136_9815.JPEG 	 19.0
images/test/n02132136/n02132136_9816.JPEG 	 13.0
images/test/n02132136/n02132136_9825.JPEG 	 41.0
images/test/n02132136/n02132136_9829.JPEG 	 38.0
images/test/n02133161/n02133161_9786.JPEG 	 20.0
images/test/n02133161/n02133161_9809.JPEG 	 36.0
images/test/n02133161/n02133161_9816.JPEG 	 41.0
images/test/n02133161/n02133161_982.JPEG 	 24.0
images/test/n02133161/n02133161_9827.JPEG 	 36.0
images/test/n02133161/n02133161_9844.JPEG 	 28.0
images/test/n02133161/n02133161_9863.JPEG 	 19.0
images/test/n02133161/n02133161_987.JPEG 	 36.0
images/test/n02133161/n02133161_9888.JPEG 	 33.0
images/test/n02134084/n

images/val/n02113023/n02113023_9902.JPEG 	 37.0
images/val/n02113023/n02113023_9906.JPEG 	 18.0
images/val/n02113023/n02113023_9920.JPEG 	 5.0
images/val/n02113023/n02113023_9926.JPEG 	 22.0
images/val/n02113023/n02113023_994.JPEG 	 24.0
images/val/n02113023/n02113023_9940.JPEG 	 27.0
images/val/n02113023/n02113023_9985.JPEG 	 13.0
images/val/n02113023/n02113023_9988.JPEG 	 27.0
images/val/n02113023/n02113023_9996.JPEG 	 38.0
images/val/n02113186/n02113186_9944.JPEG 	 6.0
images/val/n02113186/n02113186_9946.JPEG 	 4.0
images/val/n02113186/n02113186_9957.JPEG 	 41.0
images/val/n02113186/n02113186_996.JPEG 	 33.0
images/val/n02113186/n02113186_9964.JPEG 	 19.0
images/val/n02113186/n02113186_9981.JPEG 	 28.0
images/val/n02113186/n02113186_9982.JPEG 	 28.0
images/val/n02113186/n02113186_9986.JPEG 	 18.0
images/val/n02113186/n02113186_9991.JPEG 	 4.0
images/val/n02113624/n02113624_9828.JPEG 	 23.0
images/val/n02113624/n02113624_9829.JPEG 	 46.0
images/val/n02113624/n02113624_983.JPEG 	 5.0


images/val/n02123159/n02123159_9949.JPEG 	 15.0
images/val/n02123159/n02123159_998.JPEG 	 19.0
images/val/n02123159/n02123159_9983.JPEG 	 36.0
images/val/n02123159/n02123159_999.JPEG 	 5.0
images/val/n02123159/n02123159_9997.JPEG 	 19.0
images/val/n02123394/n02123394_981.JPEG 	 15.0
images/val/n02123394/n02123394_982.JPEG 	 27.0
images/val/n02123394/n02123394_983.JPEG 	 28.0
images/val/n02123394/n02123394_985.JPEG 	 28.0
images/val/n02123394/n02123394_9878.JPEG 	 33.0
images/val/n02123394/n02123394_9939.JPEG 	 5.0
images/val/n02123394/n02123394_994.JPEG 	 44.0
images/val/n02123394/n02123394_995.JPEG 	 44.0
images/val/n02123394/n02123394_996.JPEG 	 25.0
images/val/n02123597/n02123597_9888.JPEG 	 28.0
images/val/n02123597/n02123597_989.JPEG 	 13.0
images/val/n02123597/n02123597_9893.JPEG 	 26.0
images/val/n02123597/n02123597_9901.JPEG 	 41.0
images/val/n02123597/n02123597_9939.JPEG 	 38.0
images/val/n02123597/n02123597_9948.JPEG 	 28.0
images/val/n02123597/n02123597_9954.JPEG 	 4.0
image

images/val/n02165456/n02165456_9990.JPEG 	 8.0
images/val/n02165456/n02165456_9998.JPEG 	 5.0
images/val/n02167151/n02167151_9937.JPEG 	 5.0
images/val/n02167151/n02167151_9945.JPEG 	 33.0
images/val/n02167151/n02167151_9948.JPEG 	 40.0
images/val/n02167151/n02167151_9960.JPEG 	 40.0
images/val/n02167151/n02167151_9961.JPEG 	 33.0
images/val/n02167151/n02167151_9966.JPEG 	 34.0
images/val/n02167151/n02167151_9971.JPEG 	 24.0
images/val/n02167151/n02167151_9983.JPEG 	 24.0
images/val/n02167151/n02167151_9991.JPEG 	 46.0
images/val/n02168699/n02168699_9704.JPEG 	 22.0
images/val/n02168699/n02168699_9717.JPEG 	 46.0
images/val/n02168699/n02168699_9718.JPEG 	 33.0
images/val/n02168699/n02168699_974.JPEG 	 33.0
images/val/n02168699/n02168699_9758.JPEG 	 37.0
images/val/n02168699/n02168699_985.JPEG 	 45.0
images/val/n02168699/n02168699_9861.JPEG 	 19.0
images/val/n02168699/n02168699_9903.JPEG 	 19.0
images/val/n02168699/n02168699_998.JPEG 	 45.0
images/val/n02169497/n02169497_9761.JPEG 	 48.

In [6]:
#正確率
test_pre = 0
val_pre = 0

for index in range(test_num):  
    if(int(pre_test[index]) == int(test_origin[index])):
        test_pre+=1

for index in range(val_num):  
    if(int(pre_val[index]) == int(val_origin[index])):
        val_pre+=1
        
test_pre_acc = test_pre / test_num * 100
val_pre_acc = val_pre / val_num * 100
        
print('test：%.2f%%' % test_pre_acc)
print('val：%.2f%%' % val_pre_acc)

test：9.11%
val：9.56%
